In [1]:
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
!unzip train.zip
!unzip test.zip

--2025-11-26 15:32:54--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19229940 (18M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  18.34M  --.-KB/s    in 0.09s   

2025-11-26 15:32:55 (215 MB/s) - ‘train.zip’ saved [19229940/19229940]

--2025-11-26 15:32:55--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7966338 (7.6M) 

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import unicodedata
import re


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print('train', train.shape)
print('test', test.shape)

train (692500, 21)
test (296786, 20)


#### Funciones

In [9]:
def clean_data_base_2(data):
  data.drop(columns=['F_TIENEINTERNET.1', 'ID', 'PERIODO_ACADEMICO', 'E_PRIVADO_LIBERTAD'], inplace=True)

  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('ASCII'))
  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].str.replace(r'\s+', ' ', regex=True).str.strip()

  reemplazos = {
    r'\bADMINISTRACIN\b': 'ADMINISTRACION',
    r'\bLOGSTICA\b': 'LOGISTICA',
    r'\bPBLICA\b': 'PUBLICA',
    r'\bGESTIN\b': 'GESTION',
    r'\bQUMICA\b': 'QUIMICA',
    r'\bFARMACUTICA\b': 'FARMACEUTICA',
    r'\bGASTRONOMA\b': 'GASTRONOMIA',
    r'\bPSICOSOCIALES\b': 'PSICOSOCIALES',
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos, regex=True)

  reemplazos = {
    'MERCADEO':'ECONOMIA',
    'MERCADEO NACIONAL E INTERNACIONAL':'ECONOMIA',
    'MERCADEO Y VENTAS':'ECONOMIA',
    'PROFESIONAL EN MERCADEO':'ECONOMIA',
    'PROFESIONAL EN MERCADEO EMPRESARIAL':'ECONOMIA'
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos)

  return data

Se agrupan los programas academicos segun la clasificación del SNIES.
El SNIES es el Sistema Nacional de Información de la Educación Superior de Colombia.

Definición de áreas SNIES (agrupadas en 6 grupos)

In [10]:
snies_areas = {
  'SALUD': [
    r'\b(microbiolog[iaí]|bioan[áa]lisis|gerontolog[iaí]|'
    r'seguridad\s+y\s+salud\s+(en\s+el\s+trabajo|para\s+el\s+trabajo))\b',
    r'\b(medicina|enfermer[iaí]|odontolog[iaí]|fisioterapia|nutrici[óo]n|'
    r'optometr[iaí]|bacteriolog[iaí]|fonoaudiolog[iaí]|psicolog[iaí]\s+cl[ií]nica|'
    r'terapia\s+(ocupacional|cardiorrespiratoria|respiratoria)|instrumentaci[óo]n\s+quir[uú]rgica|'
    r'ciencias\s+de\s+la\s+salud|salud\s+p[úu]blica|salud\s+ocupacional|'
    r'bioquimica\s+cl[ií]nica|microbiolog[iaí]\s+y\s+bioan[áa]lisis|'
    r'farmacia|gerontolog[iaí]|seguridad\s+y\s+salud\s+en\s+el\s+trabajo)\b',
    r'\bBACTERIOLOGIA\b',
    r'\bFONOAUDIOLOGIA\b',
    r'\bODONTOLOGIA\b',
    r'\bOPTOMETRIA\b',
    r'\bENFERMERIA\b',
    r'\bMICROBIOLOG[ÍI]A\b',
    r'\bGERONTOLOG[ÍI]A\b',
    r'TERAPIAS PSICOSOCIALES'
  ],

  'INGENIERIA': [
    r'\bINGENIERIA\s+(ADMINISTRATIVA|AERONAUTICA|COMERCIAL|DE\s+MERCADOS|'
    r'DE\s+PETROLEO\s+Y\s+GAS|DE\s+PROCESOS(\s+INDUSTRIALES)?|'
    r'DE\s+PRODUCCION(\s+ACUICOLA)?|DE\s+PRODUCTIVIDAD\s+Y\s+CALIDAD|'
    r'DE\s+TELECOMUNICACIONES|EN\s+(ENERG[IAÍ]|HIGIENE\s+Y\s+SEGURIDAD\s+OCUPACIONAL|'
    r'MULTIMEDIA|PROCESOS\s+INDUSTRIALES|PRODUCCION\s+ACUICOLA|'
    r'SEGURIDAD\s+Y\s+SALUD\s+PARA\s+EL\s+TRABAJO|SOFTWARE|TELECOMUNICACIONES)|'
    r'MULTIMEDIA|TOPOGRAFICA)\b',
    r'\b(SISTEMAS|COMPUTACION|INFORMATICA|TELEMATICA|'
    r'TELECOMUNICACIONES|ELECTRONICA|ELECTRICA|INDUSTRIAL|'
    r'MECANICA|QUIMICA|AMBIENTAL|CIVIL|BIOMEDICA|SOFTWARE|'
    r'GEOLOGICA|METALURGICA|ALIMENTOS|AGRONOMICA|'
    r'MINAS|PETROLEOS|AGRICOLA|AGROECOLOGICA|AGROFORESTAL|'
    r'AGROINDUSTRIAL|BIOLOGICA|BIOQUIMICA|BIOTECNOLOGICA|'
    r'DISENO\s+DE\s+PRODUCTO|MATERIALES|SONIDO|TRANSPORTE|'
    r'ELECTROMECANICA|MECATRONICA|NANOTECNOLOGIA|TOPOGRAFIA|FORESTAL|'
    r'NAVAL|PECUARIA|PESQUERA|SANITARIA|FINANCIERA|'
    r'CONTROL|AUTOMATIZACION|INSTRUMENTACION)\b',
    r'\b(ARQUITECTURA|CONSTRUCCION|CONSTRUCCIONES\s+CIVILES)\b',
    r'\bINGENIERA|INGENIERIA\b',
    r'URBANISMO',
  ],

  'SOCIALES': [
    r'\b(derecho|jurisprudencia|psicolog[iaí](?!\s+(cl[ií]nica|ocupacional))|'
    r'comunicaci[óo]n|periodismo|relaciones\s+(internacionales|econ[óo]micas\s+internacionales)|'
    r'ciencia\s+pol[ií]tica|trabajo\s+social|antropolog[iaí]|sociolog[iaí]|'
    r'gobierno\s+y\s+asuntos\s+p[úu]blicos|planeaci[óo]n\s+y\s+desarrollo\s+social|'
    r'estudios\s+(pol[ií]ticos|literarios|musicales|sociculturales)|'
    r'criminolog[iaí]|criminal[íi]stica|investigaci[óo]n\s+criminal|'
    r'ling[uü]istica|lenguajes)\b',
    r'\bANTROPOLOGIA\b',
    r'\bSOCIOLOGIA\b',
    r'\bCIENCIAS POLITICAS\b',
    r'\bCOMUNICACIONES\b',
    r'\bLINGUISTICA\b',
    r'\bLENGUAJES Y ESTUDIOS SOCIOCULTURALES\b',
    r'\bPSICOLOGIA\b',
    r'\bPROFESIONAL EN PSICOLOGIA\b',
    r'ARQUEOLOGIA',
    r'DESARROLLO TERRITORIAL',
    r'\b(GESTION\s+CULTURAL|GESTION\s+DEPORTIVA|GESTION\s+Y\s+DESARROLLO\s+URBANOS)\b',
    r'PEDAGOGIA'
  ],

  'EDUCACION': [
    r'\b(licenciatura|pedagog[iaí]|educaci[óo]n|docencia|enseñanza|'
    r'did[áa]ctica|formaci[óo]n\s+docente|desarrollo\s+familiar)\b',
  ],

  'ECONOMIA': [
    r'MERCADEO',
    r'\b(?:MERCADEO(?: NACIONAL E INTERNACIONAL| EMPRESARIAL)?|PROFESIONAL EN MERCADEO(?: EMPRESARIAL)?)\b',
    r'\b(?:MERCADEO|PROFESIONAL\s+EN\s+MERCADEO)(?:\s+(?:NACIONAL\s+E\s+INTERNACIONAL|EMPRESARIAL))?\b',
    r'\b(administraci[óo]n|contadur[iaí]|econom[iaí]|finanzas|mercado[lt]og[íi]a|'
    r'marketing|negocios|gerencia|log[ií]stica|turismo|gesti[óo]n\s+(empresarial|'
    r'cultural\s+y\s+comunicativa|del\s+dise[ñn]o|de\s+la\s+seguridad|'
    r'de\s+mercados|financiera|urbana|territorial)|comercio\s+(exterior|internacional)|'
    r'administraci[óo]n\s+tur[íi]stica\s+y\s+hotelera|'
    r'contadur[iaí]\s+(publica|internacional)|mercadeo|econom[iaí]a)\b',
    r'\b(?:MERCADEO|MARKETING|PUBLICIDAD)(?:\s+(?:INTERNACIONAL|NACIONAL|EMPRESARIAL|Y\s+(?:PUBLICIDAD|MERCADEO|VENTAS|INTERNACIONAL)|E\s+INTERNACIONAL))?\b',
    r'\bPROFESIONAL\s+EN\s+(?:MERCADEO|MARKETING)(?:\s+(?:EMPRESARIAL|Y\s+PUBLICIDAD))?\b',
    r'\b(?:ADMINISTRACI[ÓO]N|CONTADUR[ÍI]A|ECONOM[ÍI]A|FINANZAS|'
    r'GERENCIA|LOG[ÍI]STICA|TURISMO|NEGOCIOS|COMERCIO)\b',
  ],

  'CIENCIAS': [
    r'\b(matem[áa]tica|f[íi]sica|qu[íi]mica|biolog[iaí]|geolog[iaí]|'
    r'estad[ií]stica|ciencias\s+b[áa]sicas|ecolog[iaí]|'
    r'ciencias\s+(ambientales|biol[óo]gicas|del\s+mar|n[áa]uticas|'
    r'militares\s+(aeron[áa]uticas|navales)?)|geograf[iaí]|'
    r'astrof[íi]sica|geociencias|bioingenier[iaí]a|'
    r'biotecnolog[iaí]a|bioquimica|deporte|recreaci[óo]n|'
    r'entrenamiento\s+deportivo)\b',
    r'\bBIOLOGIA\b',
    r'\bECOLOGIA\b',
    r'\bMATEMATICAS\b',
    r'\bGEOGRAFIA\b',
    r'\bGEOLOGIA\b',
    r'\bDEPORTE\b',
    r'\bRECREACION\b',
    r'\bENTRENAMIENTO DEPORTIVO\b',
    r'\bPROFESIONAL EN DEPORTE\b',
    r'\bPROFESIONAL EN ENTRENAMIENTO DEPORTIVO\b',
    r'\bCIENCIAS DEL DEPORTE\b',
    r'\bCIENCIAS DEL DEPORTE Y LA RECREACION\b',
    r'\bCIENCIAS MILITARES\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES DE INFANTERIA DE MARINA\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES NAVALES\b',
    r'ASTRONOMIA',
    r'OCEANOGRAFIA'
  ],

  'ARTES_HUMANIDADES': [
    r'\b(filosof[iaí]|historia|literatura|lenguas|traducci[óo]n|'
    r'arte|danza|dram[áa]tico|cine|audiovisuales|televisi[óo]n|'
    r'radio|producci[óo]n\s+(cinematogr[áa]fica|audiovisual|musical)|'
    r'dise[ñn]o\s+(gr[áa]fico|industrial|interior|de\s+modas|'
    r'de\s+espacios|digital|interactivo|visual|multimedia|'
    r'de\s+vestuario|escenogr[áa]fico)|gastronom[iaí]a|'
    r'culinaria|teolog[iaí]|musicolog[iaí]|interpretaci[óo]n\s+musical|'
    r'fotograf[iaí]a|bibliotecolog[iaí]a|archiv[íi]stica|'
    r'creaci[óo]n\s+literaria|espanol|filologia|'
    r'musica|formaci[óo]n\s+musical|estudios\s+(literarios|musicales|culturales))\b',
    r'\bARTES\b',
    r'\bMUSICA\b',
    r'\bFILOSOFIA\b',
    r'\bFILOSOFIA Y HUMANIDADES\b',
    r'\bFILOSOFIA Y LETRAS\b',
    r'\bESPANOL Y FILOLOGIA CLASICA\b',
    r'\bFILOLOGIA E IDIOMAS\b',
    r'\bFILOLOGIA HISPANICA\b',
    r'\bCIENCIAS BIBLICAS\b',
    r'\bTEOLOGIA\b',
    r'\bDISENO\b',
    r'\bDISENO DE MEDIOS INTERACTIVOS\b',
    r'\bDISENO Y GESTION DE LA MODA\b',
    r'\bDISENO Y PRODUCCION DE MODA\b',
    r'\bCREACION LITERARIA\b',
    r'\bESTUDIOS Y GESTION CULTURAL\b',
    r'\bCIENCIAS\s+DE\s+LA\s+INFORMACION\s+Y\s+LA\s+DOCUMENTACION\b',
    r'ANIMACION',
    r'TEATRO',
    r'DIRECCION DE BANDA',
    r'CONSERVACION Y RESTAURACION DE BIENES MUEBLES',
    r'CROSSMEDIA',
    r'NARRATIVAS DIGITALES',
  ],

  'AGROPECUARIO': [
    r'\b(agronom[iaí]|agroecolog[iaí]a|agroforestal|agroindustrial|'
    r'agropecuaria|zootecnia|acuicultura|pesquera|'
    r'producci[óo]n\s+agroindustrial|agronegocios|'
    r'profesional\s+en\s+(agroindustria|agronegocios))\b',
    r'\bAGRONOMIA\b',
    r'\bAGRONOMIA DEL TROPICO HUMEDO\b',
  ]
}


# Función para mapear programa a área
def mapper_prgm_academico_to_snies(programa):
  if not isinstance(programa, str):
      return 'OTRO'

  # Corrección de errores tipográficos frecuentes
  p = programa.upper().strip()
  p = re.sub(r'[^\w\s]', ' ', p)  # Elimina puntuación
  p = re.sub(r'\s+', ' ', p)      # Elimina espacios múltiples

  # Corección de errores de digitación del dataset
  p = re.sub(r'\bADMINSITRACION\b', 'ADMINISTRACION', p)
  p = re.sub(r'\bINTRUMENTACION\b', 'INSTRUMENTACION', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bMERCADEO\b', 'MERCADOTECNIA', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bPSICOLOGIA\b', 'PSICOLOGIA', p)

  # Busca en cada área
  for area, patrones in snies_areas.items():
      for patron in patrones:
          if re.search(patron, p, re.IGNORECASE):
              return area
  return 'OTRO'

In [11]:
def clean_prgm_departamento_to_region(data):
  region_map = {
    # Caribe
    'ATLANTICO': 'CARIBE', 'BOLIVAR': 'CARIBE', 'SUCRE': 'CARIBE', 'MAGDALENA': 'CARIBE',
    'LA GUAJIRA': 'CARIBE', 'CESAR': 'CARIBE', 'SAN ANDRES': 'CARIBE',

    # Andina
    'BOGOTÁ': 'ANDINA', 'ANTIOQUIA': 'ANDINA', 'HUILA': 'ANDINA', 'CUNDINAMARCA': 'ANDINA',
    'TOLIMA': 'ANDINA', 'VALLE': 'ANDINA', 'QUINDIO': 'ANDINA', 'RISARALDA': 'ANDINA',
    'CORDOBA': 'ANDINA', 'BOYACA': 'ANDINA', 'NARIÑO': 'ANDINA', 'CALDAS': 'ANDINA',
    'SANTANDER': 'ANDINA', 'NORTE SANTANDER': 'ANDINA',

    # Pacífica
    'CAUCA': 'PACIFICA', 'CHOCO': 'PACIFICA', 'NARIÑO': 'PACIFICA',

    # Orinoquía
    'META': 'ORINOQUIA', 'CASANARE': 'ORINOQUIA', 'ARAUCA': 'ORINOQUIA', 'GUAVIARE': 'ORINOQUIA',

    # Amazónica
    'AMAZONAS': 'AMAZONICA', 'CAQUETA': 'AMAZONICA', 'PUTUMAYO': 'AMAZONICA',
    'VAUPES': 'AMAZONICA'
  }

  data['E_PRGM_DEPARTAMENTO'] = data['E_PRGM_DEPARTAMENTO'].map(region_map).fillna('OTRO')

  return data

In [12]:
def clean_nulos_2(data):
  imputer = SimpleImputer(strategy='most_frequent')
  data['F_ESTRATOVIVIENDA'] = imputer.fit_transform(data[['F_ESTRATOVIVIENDA']])[:, 0]
  data['F_TIENEINTERNET'] = imputer.fit_transform(data[['F_TIENEINTERNET']])[:, 0]
  data['F_TIENECOMPUTADOR'] = imputer.fit_transform(data[['F_TIENECOMPUTADOR']])[:, 0]
  data['F_TIENELAVADORA'] = imputer.fit_transform(data[['F_TIENELAVADORA']])[:, 0]
  data['F_TIENEAUTOMOVIL'] = imputer.fit_transform(data[['F_TIENEAUTOMOVIL']])[:, 0]

  data['E_VALORMATRICULAUNIVERSIDAD'] = imputer.fit_transform(data[['E_VALORMATRICULAUNIVERSIDAD']])[:, 0]
  data['E_PAGOMATRICULAPROPIO'] = imputer.fit_transform(data[['E_PAGOMATRICULAPROPIO']])[:, 0]

  data['F_EDUCACIONPADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONPADRE'].transform(lambda x: x.fillna(x.mode()[0]))
  data['F_EDUCACIONMADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONMADRE'].transform(lambda x: x.fillna(x.mode()[0]))

  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].map({'0': 0, 'Menos de 10 horas': 9, 'Entre 11 y 20 horas': 15.5, 'Entre 21 y 30 horas': 25.5, 'Más de 30 horas': 31})
  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].fillna((data['E_HORASSEMANATRABAJA'].mean()).round(3))

  return data

In [13]:
def categorical_to_numeric(data):
  data = pd.get_dummies(data, columns=['E_VALORMATRICULAUNIVERSIDAD'], dtype=np.uint8)
  data.drop(columns=['E_VALORMATRICULAUNIVERSIDAD_No pagó matrícula'], inplace=True)

  data = pd.get_dummies(data, columns=['F_ESTRATOVIVIENDA'], dtype=np.uint8)
  data.drop(columns=['F_ESTRATOVIVIENDA_Sin Estrato'], inplace=True)
  data.columns = data.columns.str.replace('_Estrato ', '_')

  data['F_TIENEINTERNET'] = data['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENELAVADORA'] = data['F_TIENELAVADORA'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENEAUTOMOVIL'] = data['F_TIENEAUTOMOVIL'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['E_PAGOMATRICULAPROPIO'] = data['E_PAGOMATRICULAPROPIO'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENECOMPUTADOR'] = data['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).astype(np.int8)

  mapping = {
    'Ninguno': 0,
    'No sabe': 0,
    'No Aplica': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 7,
    'Técnica o tecnológica completa': 10,
    'Educación profesional incompleta': 13,
    'Educación profesional completa': 20,
    'Postgrado': 40
  }

  data['F_EDUCACIONMADRE'] = data['F_EDUCACIONMADRE'].map(mapping).astype(np.uint8)
  data['F_EDUCACIONPADRE'] = data['F_EDUCACIONPADRE'].map(mapping).astype(np.uint8)

  data = pd.get_dummies(data, columns=['E_PRGM_DEPARTAMENTO'], prefix='REGION', dtype=np.uint8)

  data = pd.get_dummies(data, columns=['AREA_SNIES'], dtype=np.uint8)

  return data

In [14]:
def normalize_data(data):
  data['E_HORASSEMANATRABAJA'] = (data['E_HORASSEMANATRABAJA'] - data['E_HORASSEMANATRABAJA'].min()) / (data['E_HORASSEMANATRABAJA'].max() - data['E_HORASSEMANATRABAJA'].min())
  data['F_EDUCACIONMADRE'] = (data['F_EDUCACIONMADRE'] - data['F_EDUCACIONMADRE'].min()) / (data['F_EDUCACIONMADRE'].max() - data['F_EDUCACIONMADRE'].min())
  data['F_EDUCACIONPADRE'] = (data['F_EDUCACIONPADRE'] - data['F_EDUCACIONPADRE'].min()) / (data['F_EDUCACIONPADRE'].max() - data['F_EDUCACIONPADRE'].min())

  return data

In [15]:
# Se eliman estos datos para comprobar si aportan al modelo o son contraproducentes.
def delete_Indicadors(data):
  data.drop(['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'], axis=1, inplace=True)

In [16]:
def clean_saber_pro_4(data):
  data = clean_data_base_2(data)
  data['AREA_SNIES'] = data['E_PRGM_ACADEMICO'].apply(mapper_prgm_academico_to_snies)
  data.drop('E_PRGM_ACADEMICO', axis=1, inplace=True)
  data = clean_prgm_departamento_to_region(data)
  data = clean_nulos_2(data)
  data = categorical_to_numeric(data)
  data = normalize_data(data)
  delete_Indicadors(data)

  return data

#### Main

Limpieza de los datos

In [17]:
df = train.copy()
df = clean_saber_pro_4(df)

Pasa la variable a predecir, de categorica a numérica

In [18]:
mapping = {
    'alto': 4,
    'medio-alto': 3,
    'medio-bajo': 2,
    'bajo': 1
}

df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapping).astype('int8')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 35 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   E_HORASSEMANATRABAJA                                                  692500 non-null  float64
 1   F_TIENEINTERNET                                                       692500 non-null  int8   
 2   F_EDUCACIONPADRE                                                      692500 non-null  float64
 3   F_TIENELAVADORA                                                       692500 non-null  int8   
 4   F_TIENEAUTOMOVIL                                                      692500 non-null  int8   
 5   E_PAGOMATRICULAPROPIO                                                 692500 non-null  int8   
 6   F_TIENECOMPUTADOR                                                     692500 non-nul

### Test

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

try:
    %tensorflow_version 2.x
    print ("Using TF2 in Google Colab")
except:
    pass

import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Using TF2 in Google Colab


Separo de mi dataset la variable a predecir

In [21]:
df_sample = df
X = df_sample.drop(columns=['RENDIMIENTO_GLOBAL'])
y = df_sample.RENDIMIENTO_GLOBAL

Separo una parte de los datos para testear el modelo

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")
print(f"Distribución train: {np.bincount(y_train)/len(y_train)}")

Train: (554000, 34) | Test: (138500, 34)
Distribución train: [0.         0.24980144 0.24877256 0.24782491 0.25360108]


Escala los datos, es decir, ajustar la escala de las características para que tengan media 0 y desviación estándar 1.

Las redes neuronales suelen ser sensibles a la magnitud de las características de entrada.

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(X)

Red neuronal con capa densa de entrada con 64 neuronas con activación `relu`, aplica una regulación que apaga el 30% de las neuronas para evitar sobreajuste.

Capa densa oculta con 32 neuronas con activación `relu`, y 20% de neuronas apagadas.

Capa de salida con 4 neuronas con activación `softmax`, que convierte las salidas en probabilidades para 4 clases.

Para obtimizar la precisión, se compila usando `Adam` con una tasa de aprendizaje de 0.001 y `SparseCategoricalCrossentropy` adecuada para clasificación multiclase con etiquetas enteras.

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax'),
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrena el modelo y se valida con los datos de prueba

In [25]:
history = model.fit(
    X_train_scaled, y_train - 1, validation_data=(X_test_scaled, y_test - 1),
    epochs=50, batch_size=64, verbose=1
)

Epoch 1/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.3756 - loss: 1.2921 - val_accuracy: 0.3994 - val_loss: 1.2595
Epoch 2/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.3985 - loss: 1.2620 - val_accuracy: 0.4024 - val_loss: 1.2554
Epoch 3/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.3996 - loss: 1.2594 - val_accuracy: 0.4032 - val_loss: 1.2532
Epoch 4/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.3993 - loss: 1.2594 - val_accuracy: 0.4027 - val_loss: 1.2534
Epoch 5/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4009 - loss: 1.2569 - val_accuracy: 0.4041 - val_loss: 1.2523
Epoch 6/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4014 - loss: 1.2570 - val_accuracy: 0.4042 - val_loss: 1.2527
Epoch 7/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.4018 - loss: 1.2567 - val_accuracy: 0.4042 - val_loss: 1.2523
Epoch 8/50
8657/8657 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4027 - loss: 1

Evalua el rendimiento del modelo

In [26]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train - 1, verbose=0)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test - 1, verbose=0)

print("\n📊 Resultados Red Neuronal:")
print(f"   Train Accuracy: {train_acc:.4f}")
print(f"   Test Accuracy:  {test_acc:.4f}")
print(f"   Overfitting:    {train_acc - test_acc:.4f} (ideal < 0.05)")


📊 Resultados Red Neuronal:
   Train Accuracy: 0.4068
   Test Accuracy:  0.4050
   Overfitting:    0.0019 (ideal < 0.05)


### Preparacion archivo Kaggle

Se guardan los ids de test, para al final usarlos en la creación del archivo para la submission.

En test se crea una columna que no se crea en train. Se elimina esta.

In [27]:
test = pd.read_csv('test.csv')
test_ids = test.ID
df_test = clean_saber_pro_4(test)
df_test.drop(columns=['AREA_SNIES_OTRO'], inplace=True)

In [28]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 34 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   E_HORASSEMANATRABAJA                                                  296786 non-null  float64
 1   F_TIENEINTERNET                                                       296786 non-null  int8   
 2   F_EDUCACIONPADRE                                                      296786 non-null  float64
 3   F_TIENELAVADORA                                                       296786 non-null  int8   
 4   F_TIENEAUTOMOVIL                                                      296786 non-null  int8   
 5   E_PAGOMATRICULAPROPIO                                                 296786 non-null  int8   
 6   F_TIENECOMPUTADOR                                                     296786 non-nul

Entrenamiento con todo el dataset

In [29]:
model_final = tf.keras.models.clone_model(model)
model_final.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model_final.fit(
    X_scaled, y - 1,
    epochs=30,
    batch_size=512,
    verbose=0
)

In [30]:
# Escalar test con el mismo scaler
df_test_scaled = scaler.transform(df_test)

# Predecir
y_pred_proba = model_final.predict(df_test_scaled)

# Volver a 1,2,3,4
y_pred = y_pred_proba.argmax(axis=1) + 1

9275/9275 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step


In [31]:
y_pred

array([1, 2, 4, ..., 3, 4, 4])

Pasa la variable predictora de numero a categorica

In [32]:
mapping = {
    1: "bajo",
    2: "medio-bajo",
    3: "medio-alto",
    4: "alto"
}
pred_str = pd.Series(y_pred).map(mapping).to_numpy()

In [33]:
pred_str

array(['bajo', 'medio-bajo', 'alto', ..., 'medio-alto', 'alto', 'alto'],
      dtype=object)

Generación de archivo

In [34]:
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': pred_str
})
submission.to_csv('saber_pro_2025-2_submission-2.csv', index=False)

In [35]:
!head saber_pro_2025-2_submission-2.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-bajo
499179,alto
782980,bajo
785185,bajo
58495,medio-bajo
705444,alto
557548,alto
519909,bajo
